# Assignment 4
In this task you have to scrape all the results from "Extranjero". [Here is link](https://resultadoshistorico.onpe.gob.pe/EG2021/EleccionesPresidenciales/RePres/E). 
You have to **follow the steps as we did in Lecture 7** and get information from all tabs in "Continente", "País" and "Estado".

1. Take the data table TOTAL DE VOTOS.
2. Store the four columns ORGANIZACIONES POLÍTICAS, TOTAL, %VÁLIDOS and %EMITIDOS. Rename these columns to political_parties, total, share_valid, share_emitied, respectively. 
3. Add the columns continent, country, state, ubigeo, and link. These last two items you can extract from the url. 
4. Append  the tables in just one DataFrame and export an excel file and save it in the folder [\_data\_results](https://github.com/alexanderquispe/Diplomado_PUCP/tree/main/Lecture_7/_data_results).
5. The name of your Excel file should be the same as the name of your branches. 

# <a id='outline'> Assignment 4 - Outline </a>
- <a href='#1_Lib'> 1. Libraries required </a>
- <a href='#2_web_page'> 2. Webpage </a>
- <a href='#3_scope'> 3. Scope </a>
- <a href='#4_set_up'> 4. Set Up before loop </a>
- <a href='#5_loop'> 5. Loop for webscraping </a>
- <a href='#6_final_data'> 6. Data processing and storing </a>

## <a id='1_Lib'> 1. Libraries required </a>

In [1]:
#!pip install lxml
#!pip install lxmunidecodel
#!pip install unidecode


import os
import time
import re
import unidecode
import time 
import progressbar


import pandas as pd
import numpy as np


from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

## <a id='2_web_page'> 2. Webpage </a>

One option direct link (does not work every time)

In [ ]:
#ChromeDriverManager
#driver = webdriver.Chrome( ChromeDriverManager().install() )
#driver.maximize_window()
#url = 'https://resultadoshistorico.onpe.gob.pe/EG2021/EleccionesPresidenciales/RePres/E'
#driver.get( url )
#driver.execute_script( "document.body.style.zoom='100%'" )

Second options, "ONPE Resultado Historico" link and access with clicks. May require time sleep of 10 or more seconds

In [2]:
ChromeDriverManager
driver = webdriver.Chrome( ChromeDriverManager().install() )
driver.maximize_window()
url = 'https://resultadoshistorico.onpe.gob.pe/EG2021/'
driver.get( url )
driver.execute_script( "document.body.style.zoom='100%'" )

presi = driver.find_element_by_xpath('/html/body/onpe-root/onpe-home-onpe/div[1]/div/div/div/div[2]/div[2]/div/div/a/div[2]')
presi.click()
time.sleep(10)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\josed\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


## <a id='3_scope'> 3. Scope </a>

In the part we get all the element from the scope and select our element of interest (i.e. Extranjero)

In [3]:
# Get the socpe by element_id
#scope = driver.find_element_by_id( "select_ambito" )
scope = driver.find_element_by_xpath('/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div/select')

# All selenium objects in scope
scope_options = [ x for x in scope.find_elements_by_tag_name("option") ]
dict_scope_options = { option.text : option for option in scope_options }

# We click on Extranjero
dict_scope_options['EXTRANJERO'].click()

## <a id='4_set_up'> 4. Set Up before loop </a>

In [4]:
# Store all_tables
all_tables = {}

# All selenium objects in continen selection
cnt = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select" ) )

# Get number of total options
num_cont = len( cnt.options )

## <a id='5_loop'> 5. Loop for webscraping </a>

In [ ]:
#Continent

## <a id='6_final_data'> 6. Data processing and storing </a>

In [ ]:
final_data = pd.concat( all_tables.values() ).reset_index( drop = True )
final_data = final_data.rename( columns={'organizaciones_politicas': ' political_parties',
                                         'share_validos': 'share_valid',
                                         'share_emitidos': 'share_emitied'})
final_data.to_excel( r'../_data_results/group_5_ass_4.xlsx' , index = False )
final_data